In [20]:
from keras.models import Sequential
from keras.layers import Dropout, GlobalAveragePooling1D, Embedding,Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import json
import numpy as np
import pickle

In [21]:
with open('dataset.json','r') as outputfile:
    dataset=json.load(outputfile)

In [22]:
patterns,intents,tags=[],[],[]

In [23]:
for intent in dataset['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        intents.append(intent['tag'])
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

In [24]:
num={}
for idx,tag in enumerate(tags):
    num[tag]=idx

In [25]:
with open('num.pickle', 'wb') as handle:
    pickle.dump(num, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
idx_intents=[]
for intent in intents:
    idx_intents.append(num[intent])


In [27]:
tokenizer = Tokenizer(oov_token='<OOV>')

In [28]:
tokenizer.fit_on_texts(patterns)

In [29]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
total_words=len(tokenizer.word_index)+1

In [31]:
sequences=tokenizer.texts_to_sequences(patterns)

In [32]:
sequences

[[43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49, 3, 50],
 [51],
 [21],
 [52, 3],
 [53, 22],
 [21, 4, 54],
 [5, 9, 23],
 [14, 3, 24, 4, 23],
 [55, 11, 3],
 [25, 56, 11, 3],
 [5, 9, 57],
 [58, 59, 3, 60],
 [5, 8, 26, 27, 61],
 [6, 2, 7, 8, 26],
 [62, 22, 63, 28, 64],
 [6, 2, 4, 15],
 [29, 2, 4, 15],
 [15, 65],
 [30, 2],
 [31, 66, 67],
 [68, 3, 30],
 [16, 32, 33],
 [5, 9, 17, 69],
 [70, 32, 33],
 [6, 2, 8, 34, 12],
 [5, 35, 18, 8, 36],
 [10, 12, 71],
 [16, 4, 19],
 [5, 4, 37, 19, 28, 72],
 [6, 2, 7, 4, 19],
 [6, 2, 7, 73],
 [5, 8, 34, 18, 38],
 [10, 38, 12],
 [6, 2, 7, 20],
 [5, 35, 18, 8, 20, 36],
 [10, 20, 12],
 [13, 74],
 [13, 75],
 [37, 76],
 [13, 77],
 [13, 78],
 [31, 79, 27, 3],
 [6, 2, 80, 39],
 [29, 2, 4, 39, 40],
 [81, 2],
 [25, 11, 3, 82],
 [11, 3, 83],
 [14, 3, 24, 84],
 [6, 2, 7, 41],
 [85, 41, 86, 14, 3, 87],
 [10, 88, 89],
 [6, 2, 7, 42],
 [5, 9, 17, 90, 40],
 [10, 91, 42, 92],
 [16, 4, 93],
 [5, 9, 17, 94],
 [6, 2, 7, 4, 95, 96]]

In [33]:
padded_sequence=pad_sequences(sequences)

In [34]:
padded_sequence.shape[1]

6

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequence, np.array(idx_intents), test_size=0.2, random_state=42)

In [37]:
idx_intents

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 10,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 15,
 15,
 15,
 16,
 16,
 16,
 17,
 17,
 17,
 18,
 18,
 18,
 19,
 19,
 19]

In [38]:
model = Sequential([
    Embedding(total_words, 16, input_length=padded_sequence.shape[1]),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(len(set(intents)), activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
history=model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))

Epoch 1/1000


2/2 [==============================] - 0s 210ms/step - loss: 2.9797 - accuracy: 0.3400 - val_loss: 2.9947 - val_accuracy: 0.0769
Epoch 2/1000
2/2 [==============================] - 0s 47ms/step - loss: 2.9780 - accuracy: 0.3600 - val_loss: 2.9947 - val_accuracy: 0.0769
Epoch 3/1000
2/2 [==============================] - 0s 51ms/step - loss: 2.9762 - accuracy: 0.3800 - val_loss: 2.9947 - val_accuracy: 0.0769
Epoch 4/1000
2/2 [==============================] - 0s 74ms/step - loss: 2.9743 - accuracy: 0.3800 - val_loss: 2.9949 - val_accuracy: 0.0769
Epoch 5/1000
2/2 [==============================] - 0s 67ms/step - loss: 2.9725 - accuracy: 0.4200 - val_loss: 2.9951 - val_accuracy: 0.0769
Epoch 6/1000
2/2 [==============================] - 0s 55ms/step - loss: 2.9705 - accuracy: 0.4600 - val_loss: 2.9952 - val_accuracy: 0.0769
Epoch 7/1000
2/2 [==============================] - 0s 71ms/step - loss: 2.9686 - accuracy: 0.4600 - val_loss: 2.9953 - val_accuracy: 0.0769
Epoch 8/1000
2/2 [=======

In [41]:
model.save('training.h5')

c:\Users\SAMRAT KUMAR DAS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
abc={'a':1,
     'b':2}

In [49]:
print(abc.get('a'))

1
